In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


## Let's load the training data

In [2]:
summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')
prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')

## Check dataframe description

In [3]:
summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7165 entries, 0 to 7164
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   student_id  7165 non-null   object 
 1   prompt_id   7165 non-null   object 
 2   text        7165 non-null   object 
 3   content     7165 non-null   float64
 4   wording     7165 non-null   float64
dtypes: float64(2), object(3)
memory usage: 280.0+ KB


In [4]:
prompt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prompt_id        4 non-null      object
 1   prompt_question  4 non-null      object
 2   prompt_title     4 non-null      object
 3   prompt_text      4 non-null      object
dtypes: object(4)
memory usage: 256.0+ bytes


## Primary key to combine dataframe on

In [5]:
prompt_df['prompt_id'].unique()

array(['39c16e', '3b9047', '814d6b', 'ebad26'], dtype=object)

In [6]:
summary_df['prompt_id'].unique()

array(['814d6b', 'ebad26', '3b9047', '39c16e'], dtype=object)

## Merge dataframe based on primary key into one

In [7]:
train_df = summary_df.merge(right=prompt_df,how='inner', on='prompt_id')

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7165 entries, 0 to 7164
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   student_id       7165 non-null   object 
 1   prompt_id        7165 non-null   object 
 2   text             7165 non-null   object 
 3   content          7165 non-null   float64
 4   wording          7165 non-null   float64
 5   prompt_question  7165 non-null   object 
 6   prompt_title     7165 non-null   object 
 7   prompt_text      7165 non-null   object 
dtypes: float64(2), object(6)
memory usage: 503.8+ KB


### Let's perform a little exploratory analysis on the data to see what samples looks like

In [9]:
print('''Question: %s \n 
Title: %s \n
Text: %s'''%(train_df.iloc[1]['prompt_question'],train_df.iloc[1]['prompt_title'],train_df.iloc[1]['prompt_text']))

Question: Summarize how the Third Wave developed over such a short period of time and why the experiment was ended. 
 
Title: The Third Wave 

Text: Background 
The Third Wave experiment took place at Cubberley High School in Palo Alto, California during the first week of April 1967. History teacher Ron Jones, finding himself unable to explain to his students how people throughout history followed the crowd even when terrible things were happening, decided to demonstrate it to his students through an experiment. Jones announced that he was starting a movement aimed to eliminate democracy. Jones named the movement “The Third Wave” as a symbol of strength, referring to the mythical belief that the third in a series of waves is the strongest. One of the central points of this movement was that democracy’s main weakness is that it favors the individual over the whole community. Jones emphasized this main point of the movement when he created this catchy motto: “Strength through discipline,

### Let's check what the student summary looks like

In [10]:
from pprint import pformat
print(pformat('''Question: %s
Summary: %s
Content Score: %s
Wording Score: %s'''%(train_df.iloc[0]['prompt_question'],train_df.iloc[0]['text'], train_df.iloc[0]['content'],
                           train_df.iloc[0]['wording'])))

('Question: Summarize how the Third Wave developed over such a short period of '
 'time and why the experiment was ended.\n'
 'Summary: The third wave was an experimentto see how people reacted to a new '
 'one leader government. It gained popularity as people wanted to try new '
 'things. The students follow anything that is said and start turning on '
 'eachother to gain higher power. They had to stop the experement as too many '
 'people got to radical with it blindly following there leader\n'
 'Content Score: 0.205682506482641\n'
 'Wording Score: 0.380537638762288')


## Let's build a helper function that displays N text in a pretty format

In [11]:
def display_text(s=1,n=10,df=train_df):
    """
    This function takes n as input and displays n section of text
    """
    for i in range(s,n):
        p_sum = pformat('''Question: %s \nSummary: %s \nContent Score: %s \nWording Score: %s'''%(df.iloc[i]['prompt_question'],df.iloc[i]['text'], df.iloc[i]['content'],
                           df.iloc[i]['wording']))
        print(p_sum,end='\n\n')

In [12]:
display_text(s=200,n=220,df=train_df)

('Question: Summarize how the Third Wave developed over such a short period of '
 'time and why the experiment was ended. \n'
 'Summary: The Third Wave developed over such a short period of time. Jones '
 'emphasized many simple rules and made sure the students did it right. '
 'Students also had to salute each other in a specific way that the teacher '
 'had made. On the fourth day, Jones claimed that things started getting out '
 'of control because the students became increasingly involved in the project, '
 'so their discipline and loyalty to the project was overly outstanding. So '
 'Jones decided to terminate the movement and lied to the students by '
 'announcing that the Third Wave was a part of a nationwide movement.  \n'
 'Content Score: 1.46743085810556 \n'
 'Wording Score: 1.84017156306968')

('Question: Summarize how the Third Wave developed over such a short period of '
 'time and why the experiment was ended. \n'
 'Summary: It happened so fastly because the orig nail kid

#### Combine the Title, Question, Context and Summary text into one large block of text

In [13]:
sample = 'Title:\n'+train_df.iloc[0]['prompt_title']+'\nQuestion:\n'+train_df.iloc[0]['prompt_question']+'\nText:\n'+train_df.iloc[0]['prompt_text']+'\nAnswer:\n'+train_df.iloc[0]['text']

In [14]:
print(sample)

Title:
The Third Wave
Question:
Summarize how the Third Wave developed over such a short period of time and why the experiment was ended.
Text:
Background 
The Third Wave experiment took place at Cubberley High School in Palo Alto, California during the first week of April 1967. History teacher Ron Jones, finding himself unable to explain to his students how people throughout history followed the crowd even when terrible things were happening, decided to demonstrate it to his students through an experiment. Jones announced that he was starting a movement aimed to eliminate democracy. Jones named the movement “The Third Wave” as a symbol of strength, referring to the mythical belief that the third in a series of waves is the strongest. One of the central points of this movement was that democracy’s main weakness is that it favors the individual over the whole community. Jones emphasized this main point of the movement when he created this catchy motto: “Strength through discipline, stre

In [15]:
train_df['full_text'] = train_df.apply(lambda x: 'Title:\n'+x['prompt_title']+
               '\nQuestion:\n'+x['prompt_question']+
               '\nText:\n'+x['prompt_text']+
               '\nAnswer:\n'+x['text'],axis=1)

### Make a copy of the dataframe

In [16]:
df = train_df[['student_id','full_text','content','wording']].copy()

In [17]:
df.head(2)

,student_id,full_text,content,wording
0,000e8c3c7ddb,Title:\nThe Third Wave\nQuestion:\nSummarize h...,0.205683,0.380538
1,0070c9e7af47,Title:\nThe Third Wave\nQuestion:\nSummarize h...,3.272894,3.219757


### Let's perform text preprocessing on our df full_text column

In [18]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Remove Stopwords

In [19]:
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer #,PorterStemmer, WordNetLemmatizer

In [20]:
def clean_text(text):
    stops = stopwords.words('english')
    processed_text = []
    for word in text.split():
        if word not in stops:
            processed_text.append(word)
    return ' '.join(processed_text)

In [21]:
df['full_text'] = df['full_text'].apply(clean_text)

### Convert words to stem

In [22]:
def stem_word(text):
    stem = LancasterStemmer()
    stemmed_data = []
    for word in text.split():
        stemmed_data.append(stem.stem(word.lower()))
    return ' '.join(stemmed_data)

In [23]:
df['full_text'] = df['full_text'].apply(stem_word)

### Create Bag of Words for data

In [24]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [25]:
vectorizer = CountVectorizer(ngram_range=(1,1))
transformer = TfidfTransformer()

In [26]:
transformed_df = vectorizer.fit_transform(df['full_text'])

In [27]:
vocab = list(vectorizer.vocabulary_.keys())

In [28]:
bags = pd.DataFrame(transformed_df.toarray(), columns=vocab,index=df['student_id'])

In [29]:
df_bag = bags.merge(df.set_index(keys='student_id')[['content','wording']], on='student_id').values

### Split train data into train and val data

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train,X_test,y_train,y_test = train_test_split(df_bag[:,:-2] ,df_bag[:,-2:],test_size=0.1,shuffle=True,random_state=11) 

### Train multi-output algorithm with scikit learn multi-regressor

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV

In [33]:
def mcrmse(y_true, y_pred):
    return np.mean(np.sqrt(np.mean((y_true - y_pred)**2, axis=0)))

mcrmse_scorer = make_scorer(mcrmse, greater_is_better=False)

In [34]:
model_rf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,criterion='squared_error',max_depth=5,random_state=11),n_jobs=-1)

In [35]:
model_rf.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=5,
                                                     random_state=11),
                     n_jobs=-1)

In [36]:
y_preds = model_rf.predict(X_test)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [37]:
mcrmse_scorer(model_rf,X_test,y_test)

-0.8241141260612206

In [38]:
mcrmse(y_test,y_preds)

0.8241141260612206

In [39]:
test_summary = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
test_prompts = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')

### Combine all text preprocessing into a function

In [40]:
def preprocess_data(summary_df, prompt_df, vectorizer=vectorizer):
    test_df = summary_df.merge(right=prompt_df,how='inner', on='prompt_id')
    test_df['full_text'] = test_df.apply(lambda x: 'Title:\n'+x['prompt_title']+
               '\nQuestion:\n'+x['prompt_question']+
               '\nText:\n'+x['prompt_text']+
               '\nAnswer:\n'+x['text'],axis=1)
    df = test_df[['student_id','full_text']].copy()
    df['full_text'] = df['full_text'].apply(clean_text)
    df['full_text'] = df['full_text'].apply(stem_word)
    transformed_df = vectorizer.transform(df['full_text'])
    vocab = list(vectorizer.vocabulary_.keys())
    bags = pd.DataFrame(transformed_df.toarray(), columns=vocab,index=df['student_id'])
    return bags

In [41]:
test_df = preprocess_data(test_summary, test_prompts)

In [42]:
y_preds = model_rf.predict(test_df.values)

In [43]:
df_test = pd.DataFrame(y_preds,columns=['content','wording'],index=test_df.index).reset_index()

In [44]:
df_test.to_csv("submission.csv", index=False)